In [1]:
!pip install datasets transformers tokenizers

In [2]:
!pip install seqeval -q

In [6]:
!pip install --upgrade setuptools pip

  Using cached setuptools-68.0.0-py3-none-any.whl.metadata (6.4 kB)
Using cached setuptools-68.0.0-py3-none-any.whl (804 kB)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 41.2.0
    Uninstalling setuptools-41.2.0:
      Successfully uninstalled setuptools-41.2.0


In [3]:
!pip install accelerate -U

In [4]:
!pip install transformers[torch]

In [1]:
import datasets
import numpy
from transformers import BertTokenizerFast
from transformers import DataCollatorForTokenClassification
from transformers import AutoModelForTokenClassification

C:\Users\Dr.G.Mahadevan\AppData\Local\Programs\Python\Python37\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = datasets.load_dataset("conll2003")

Found cached dataset conll2003 (C:/Users/Dr.G.Mahadevan/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98)
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 20.94it/s]


In [3]:
dataset['train'][0]

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

In [4]:
example = dataset['train'][0:5]
example['ner_tags']

[[3, 0, 7, 0, 0, 0, 7, 0, 0],
 [1, 2],
 [5, 0],
 [0,
  3,
  4,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  0,
  0,
  0,
  0,
  0,
  7,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [5,
  0,
  0,
  0,
  0,
  3,
  4,
  0,
  0,
  0,
  1,
  2,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  5,
  0,
  0,
  0,
  0,
  0,
  0,
  0]]

In [5]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

In [6]:
def tokenize_and_align_labels(examples, label_all_tokens = True):
    tokenized_inputs = tokenizer(examples['tokens'],truncation = True, is_split_into_words = True)
    labels = []
    for i,label in enumerate(examples['ner_tags']):   
        word_ids = tokenized_inputs.word_ids(batch_index = i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [7]:
eg = dataset['train'][4:5]

In [8]:
eg

{'id': ['4'],
 'tokens': [['Germany',
   "'s",
   'representative',
   'to',
   'the',
   'European',
   'Union',
   "'s",
   'veterinary',
   'committee',
   'Werner',
   'Zwingmann',
   'said',
   'on',
   'Wednesday',
   'consumers',
   'should',
   'buy',
   'sheepmeat',
   'from',
   'countries',
   'other',
   'than',
   'Britain',
   'until',
   'the',
   'scientific',
   'advice',
   'was',
   'clearer',
   '.']],
 'pos_tags': [[22,
   27,
   21,
   35,
   12,
   22,
   22,
   27,
   16,
   21,
   22,
   22,
   38,
   15,
   22,
   24,
   20,
   37,
   21,
   15,
   24,
   16,
   15,
   22,
   15,
   12,
   16,
   21,
   38,
   17,
   7]],
 'chunk_tags': [[11,
   11,
   12,
   13,
   11,
   12,
   12,
   11,
   12,
   12,
   12,
   12,
   21,
   13,
   11,
   12,
   21,
   22,
   11,
   13,
   11,
   1,
   13,
   11,
   17,
   11,
   12,
   12,
   21,
   1,
   0]],
 'ner_tags': [[5,
   0,
   0,
   0,
   0,
   3,
   4,
   0,
   0,
   0,
   1,
   2,
   0,
   0,
   0,
   0,
   0,


In [9]:
q = tokenize_and_align_labels(eg)
q

{'input_ids': [[101, 2762, 1005, 1055, 4387, 2000, 1996, 2647, 2586, 1005, 1055, 15651, 2837, 14121, 1062, 9328, 5804, 2056, 2006, 9317, 10390, 2323, 4965, 8351, 4168, 4017, 2013, 3032, 2060, 2084, 3725, 2127, 1996, 4045, 6040, 2001, 24509, 1012, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[-100, 5, 0, 0, 0, 0, 0, 3, 4, 0, 0, 0, 0, 1, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, -100]]}

In [10]:
for token,label in zip(tokenizer.convert_ids_to_tokens(q['input_ids'][0]),q['labels'][0]):
    print(f"{token:-<40}  {label}")

[CLS]-----------------------------------  -100
germany---------------------------------  5
'---------------------------------------  0
s---------------------------------------  0
representative--------------------------  0
to--------------------------------------  0
the-------------------------------------  0
european--------------------------------  3
union-----------------------------------  4
'---------------------------------------  0
s---------------------------------------  0
veterinary------------------------------  0
committee-------------------------------  0
werner----------------------------------  1
z---------------------------------------  2
##wing----------------------------------  2
##mann----------------------------------  2
said------------------------------------  0
on--------------------------------------  0
wednesday-------------------------------  0
consumers-------------------------------  0
should----------------------------------  0
buy--------------------------

In [11]:
tokenized_dataset = dataset.map(tokenize_and_align_labels, batched = True)

Loading cached processed dataset at C:\Users\Dr.G.Mahadevan\.cache\huggingface\datasets\conll2003\conll2003\1.0.0\9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98\cache-5779b3e5f06a8303.arrow
Loading cached processed dataset at C:\Users\Dr.G.Mahadevan\.cache\huggingface\datasets\conll2003\conll2003\1.0.0\9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98\cache-49336a601559a173.arrow
Loading cached processed dataset at C:\Users\Dr.G.Mahadevan\.cache\huggingface\datasets\conll2003\conll2003\1.0.0\9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98\cache-a1fc168ae24c007c.arrow


In [12]:
tokenized_dataset['train'][0]

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0],
 'input_ids': [101,
  7327,
  19164,
  2446,
  2655,
  2000,
  17757,
  2329,
  12559,
  1012,
  102],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'labels': [-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, -100]}

In [13]:
dataset['train'][0]

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

In [14]:
model = AutoModelForTokenClassification.from_pretrained("bert-base-uncased",num_labels = 9)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: 

In [15]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    "test-ner",
    evaluation_strategy = "epoch",
    learning_rate = 2e-5,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 16,
    num_train_epochs = 3,
    weight_decay = 0.01
)

In [16]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [17]:
metric = datasets.load_metric("seqeval")

C:\Users\Dr.G.Mahadevan\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  """Entry point for launching an IPython kernel.


In [18]:
example = dataset['train'][0]

In [19]:
label_list = dataset['train'].features['ner_tags'].feature.names
label_list

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [20]:
labels = [label_list[i] for i in example['ner_tags']]

In [21]:
labels

['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O']

In [22]:
metric.compute(predictions=[labels],references=[labels])

{'MISC': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'ORG': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

In [23]:
import numpy as np
def compute_metrics(eval_preds):
    logits, labels = eval_preds
    logits = np.argmax(logits,axis=2)
    predictions = [
        [label_list[eval_preds] for (eval_preds,l) in zip(prediction,label)  if l != -100]
        for prediction,label in zip(logits,labels)
    ]
    
    true_labels = [
        [label_list[l] for (eval_preds,l) in zip(prediction,label)  if l != -100]
        for prediction,label in zip(logits,labels)
    ]
    
    results = metric.compute(predictions=predictions,references=true_labels)
    
    return {
        "precision":results['overall_precision'],
        "recall":results['overall_recall'],
        "f1":results['overall_f1'],
        "accuracy":results['overall_accuracy']
    }
    
    

In [24]:
trainer = Trainer(
    model,
    args,
    train_dataset = tokenized_dataset['train'],
    eval_dataset = tokenized_dataset['validation'],
    data_collator = data_collator,
    tokenizer = tokenizer,
    compute_metrics = compute_metrics
)

In [25]:
trainer.train()

C:\Users\Dr.G.Mahadevan\AppData\Local\Programs\Python\Python37\lib\site-packages\transformers\optimization.py:415: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.218000,0.061485,0.925938,0.930082,0.928005,0.983478
2,0.050100,0.053564,0.929447,0.941716,0.935541,0.985496
3,0.024600,0.055377,0.934672,0.942723,0.938680,0.985655


TrainOutput(global_step=2634, training_loss=0.07766928605808845, metrics={'train_runtime': 36526.6942, 'train_samples_per_second': 1.153, 'train_steps_per_second': 0.072, 'total_flos': 1021316467278600.0, 'train_loss': 0.07766928605808845, 'epoch': 3.0})

In [26]:
model.save_pretrained("NER-model")

In [27]:
tokenizer.save_pretrained("Tokenizer")

('Tokenizer\\tokenizer_config.json',
 'Tokenizer\\special_tokens_map.json',
 'Tokenizer\\vocab.txt',
 'Tokenizer\\added_tokens.json',
 'Tokenizer\\tokenizer.json')

In [28]:
id2label = { str(i):label for i,label in enumerate(label_list) }
label2id = { label:str(i) for i,label in enumerate(label_list) }

In [29]:
id2label

{'0': 'O',
 '1': 'B-PER',
 '2': 'I-PER',
 '3': 'B-ORG',
 '4': 'I-ORG',
 '5': 'B-LOC',
 '6': 'I-LOC',
 '7': 'B-MISC',
 '8': 'I-MISC'}

In [30]:
label2id

{'O': '0',
 'B-PER': '1',
 'I-PER': '2',
 'B-ORG': '3',
 'I-ORG': '4',
 'B-LOC': '5',
 'I-LOC': '6',
 'B-MISC': '7',
 'I-MISC': '8'}

In [32]:
import json
config = json.load(open("NER-model/config.json"))
config['id2label'] = id2label
config['label2id'] = label2id
json.dump(config,open("NER-model/config.json","w"))

In [33]:
fine_tuned_model = AutoModelForTokenClassification.from_pretrained("NER-model")

In [34]:
from transformers import pipeline
pipe = pipeline("ner",model = fine_tuned_model, tokenizer = tokenizer)

In [39]:
sample_text = "My name is Ashwinth and I am a Computer Science student at IIITDM"
# sample_text = "Bill gates is the founder of Microsoft"
res = pipe(sample_text)
for entity in res:
    print(res)

[{'entity': 'B-PER', 'score': 0.99755883, 'index': 4, 'word': 'ash', 'start': 11, 'end': 14}, {'entity': 'B-PER', 'score': 0.997781, 'index': 5, 'word': '##win', 'start': 14, 'end': 17}, {'entity': 'B-PER', 'score': 0.9971125, 'index': 6, 'word': '##th', 'start': 17, 'end': 19}, {'entity': 'B-ORG', 'score': 0.95637214, 'index': 15, 'word': 'iii', 'start': 59, 'end': 62}, {'entity': 'B-ORG', 'score': 0.9333498, 'index': 16, 'word': '##t', 'start': 62, 'end': 63}, {'entity': 'B-ORG', 'score': 0.96373814, 'index': 17, 'word': '##dm', 'start': 63, 'end': 65}]
[{'entity': 'B-PER', 'score': 0.99755883, 'index': 4, 'word': 'ash', 'start': 11, 'end': 14}, {'entity': 'B-PER', 'score': 0.997781, 'index': 5, 'word': '##win', 'start': 14, 'end': 17}, {'entity': 'B-PER', 'score': 0.9971125, 'index': 6, 'word': '##th', 'start': 17, 'end': 19}, {'entity': 'B-ORG', 'score': 0.95637214, 'index': 15, 'word': 'iii', 'start': 59, 'end': 62}, {'entity': 'B-ORG', 'score': 0.9333498, 'index': 16, 'word': '##